In [1]:
%%capture
# Установим библиотеку трансформеров
!pip install transformers

In [32]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# Импортируем библиотеки
from transformers import AutoModelForSeq2SeqLM, T5TokenizerFast

In [3]:
# Зададим название выбронной модели из хаба
MODEL_NAME = 'UrukHan/t5-russian-spell'
MAX_INPUT = 256

In [4]:
# Загрузка модели и токенизатора
tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [5]:
import pandas as pd
df = pd.read_csv('dataset.csv')
df.head()

,Unnamed: 0,source,target
0,0,очень классная тетка ктобы что не говорил.,очень классная тетка кто бы что ни говорил
1,1,Может выгоднее втулку продать и купить колесо ...,Может выгоднее втулку продать и купить колесо ...
2,2,Довольно большая часть пришедших сходила с дор...,Довольно большая часть пришедших сходила с дор...
3,3,"Симпатичнейшое шпионское устройство, такой себ...",Симпатичнейшее шпионское устройство такой себе...
4,4,Опофеозом дня для меня сегодня стала фраза усл...,Апофеозом дня для меня сегодня стала фраза усл...


In [17]:
# Входные данные (можно массив фраз или текст)
input_sequences = list(df.source.values)
# можно использовать одиночные фразы: 
# input_sequences = 'сеглдыя хорош ден'
input_sequences = input_sequences[:2000]

In [31]:
del model

In [26]:
# Токенизирование данных
task_prefix = "Spell correct: "
if type(input_sequences) != list: input_sequences = [input_sequences]
encoded = tokenizer(
  [task_prefix + sequence for sequence in input_sequences],
  padding="longest",
  max_length=MAX_INPUT,
  truncation=True,
  return_tensors="pt",
)

In [8]:
import torch

In [9]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [10]:
device
model = model.to(device)

In [18]:
test_loader = torch.utils.data.DataLoader(input_sequences , batch_size=64)

In [22]:
predicts = []
for input_sequences in test_loader:
    task_prefix = "Spell correct: "
    if type(input_sequences) != list: input_sequences = [input_sequences]
    encoded = tokenizer(
      [task_prefix + sequence for sequence in input_sequences],
      padding="longest",
      max_length=MAX_INPUT,
      truncation=True,
      return_tensors="pt",
    )
    predicts_ = model.generate(**encoded.to(device))
    predicts_= tokenizer.batch_decode(predicts_, skip_special_tokens=True);
    predicts += predicts_

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1301: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 256 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [25]:
len(predicts)

2000

In [77]:
targets = df.target[:2000]
targets = list(map(clear, targets))

In [60]:
import nltk
import string


def clear (s):
  punct = "!\"#$%&'()*+,./:;<=>?@[\]^_`{|}~'"
  tokens = nltk.word_tokenize(s)
  tokens = list(filter(lambda token: token not in string.punctuation, tokens))
  s = ' '.join(tokens)
  return s.translate(str.maketrans('', '', punct)).lower()


In [61]:
predicts = list(map(clear, predicts))

In [76]:
accuracy_score = 0
for i in range(2000):
  accuracy_score += int(predicts[i] == targets[i])
print(accuracy_score/20)

64.55
